In [1]:
import pandas as pd
import numpy as np
from datetime import *
from glob import glob

In [2]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

print(tf.__version__)

2.1.0


In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import r2_score

In [4]:
import pylab as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

## Get data

In [5]:
with open('pipeline_data/steps.dat', 'r') as handle:
    steps = handle.readline().strip().split()

In [6]:
csv_fname = glob('pipeline_data/data_????-??-??_glm.csv')[0]
df = pd.read_csv(csv_fname, dtype={'fips': str}).set_index('fips')
df.head()

,case0,case1,case2,case3,case4,case5,case6,case7,case8,case9,...,death24_glm,death25_glm,death26_glm,death27_glm,death28_glm,death29_glm,death30_glm,death31_glm,death32_glm,death33_glm
fips,,,,,,,,,,,,,,,,,,,,,
01001,12.0,7.0,6.0,11.0,11.0,16.0,45.0,48.0,60.0,48.0,...,1.63,1.36,1.39,1.51,1.41,1.89,1.93,2.58,2.54,3.32
01003,34.0,36.0,40.0,45.0,30.0,34.0,42.0,19.0,12.0,29.0,...,2.66,2.42,2.17,2.28,2.69,3.30,3.21,3.52,4.07,5.29
01005,3.0,7.0,8.0,16.0,9.0,14.0,24.0,27.0,47.0,41.0,...,1.44,1.01,0.89,0.89,0.66,0.71,0.58,1.07,0.68,0.85
01007,4.0,9.0,13.0,8.0,7.0,4.0,7.0,7.0,15.0,5.0,...,0.80,0.64,0.68,0.76,0.61,0.79,0.84,1.30,1.06,1.32
01009,12.0,3.0,9.0,12.0,6.0,5.0,2.0,3.0,13.0,11.0,...,0.71,0.54,0.57,0.62,0.64,0.80,0.89,1.08,1.07,1.22


## Functions

In [7]:
class tfRegr:
    
    def __init__(self, 
                 epoch=100, 
                 verbose=False, 
                 validation_split=0.2, 
                 learning_rate=.1):
        
        self.normalizer = preprocessing.Normalization()
        self.ep = epoch
        self.vb = verbose
        self.vs = validation_split
        self.lr = learning_rate
    
    def fit(self, X, y):
        self.normalizer.adapt(X)

        self.model = tf.keras.Sequential([
            self.normalizer,
            layers.Dense(units=1)
        ])

        self.model.compile(
            optimizer=tf.optimizers.Adam(learning_rate=self.lr),
            loss='mean_absolute_error'
        )

        self.history = self.model.fit(
            X, y,
            epochs=self.ep,
            verbose=self.vb, # logging
            validation_split=self.vs # Calculate validation results on a fraction of the training data
        )
        return self.model

In [8]:
def predict(df, cur_idx):

    regrs = {
        'RF': RandomForestRegressor(min_samples_split=2), 
        'ET': ExtraTreesRegressor(min_samples_split=2), 
        'TF': tfRegr(),
    }
    
    cur = f'case{cur_idx}'

    glm = [f'case{cur_idx - 1}_glm', f'case{cur_idx}_glm'] 
    cols = [f'case{cur_idx - 1}'] + glm
    X, y = df[cols].values, df[cur].values 
    
    
    for name, regr in regrs.items():
        regr = regr.fit(X, y)

        #================ Evaluation ===============START
        y_pred = regr.predict(X).flatten()
        r2 = r2_score(y, y_pred)
        rmse = np.sqrt(((y - y_pred) ** 2).mean())
        print(f'\t{name}: r2 = {r2:.3f}, rmse = {rmse:.3f}')
        #================ Evaluation ===============START
        
        cols = [cur] + glm
        df[cur + f'_{name}_1'] = regr.predict(df[cols].values).flatten()
        
        cols = [cur + f'_{name}_1'] + glm
        df[cur + f'_{name}_2'] = regr.predict(df[cols].values).flatten()
    
    prd_cols_1 = [col for col in df.columns if col.endswith('_1')]
    prd_cols_2 = [col for col in df.columns if col.endswith('_2')]
    prd_1 = df[prd_cols_1].mean(axis=1).values
    prd_2 = df[prd_cols_2].mean(axis=1).values
    
    df.drop(prd_cols_1 + prd_cols_2, axis=1, inplace=True)
    
    return prd_1, prd_2

## Forecast Runs

In [9]:
cur_idx = len(steps) - 1
print(steps[cur_idx])

num_runs = 100
prds_1, prds_2 = [], []

for r in range(num_runs):
    print(f'run = {r}')
    prd_1, prd_2 = predict(df, cur_idx)
    prds_1.append(prd_1)
    prds_2.append(prd_2)

2020-11-21
run = 0
	RF: r2 = 0.974, rmse = 181.040
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.910, rmse = 335.961
run = 1
	RF: r2 = 0.966, rmse = 206.049
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.910, rmse = 335.921
run = 2
	RF: r2 = 0.980, rmse = 157.083
	ET: r2 = 1.000, rmse = 0.062
	TF: r2 = 0.911, rmse = 334.929
run = 3
	RF: r2 = 0.978, rmse = 166.861
	ET: r2 = 1.000, rmse = 0.078
	TF: r2 = 0.911, rmse = 334.520
run = 4
	RF: r2 = 0.964, rmse = 212.372
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.910, rmse = 335.645
run = 5
	RF: r2 = 0.972, rmse = 189.066
	ET: r2 = 1.000, rmse = 0.064
	TF: r2 = 0.911, rmse = 333.949
run = 6
	RF: r2 = 0.978, rmse = 165.929
	ET: r2 = 1.000, rmse = 0.005
	TF: r2 = 0.908, rmse = 339.391
run = 7
	RF: r2 = 0.981, rmse = 153.883
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.909, rmse = 337.817
run = 8
	RF: r2 = 0.979, rmse = 161.483
	ET: r2 = 1.000, rmse = 0.060
	TF: r2 = 0.911, rmse = 334.656
run = 9
	RF: r2 = 0.977, rmse = 168.856
	ET: r2 = 1.000, rmse = 0.000
	

	TF: r2 = 0.911, rmse = 334.938
run = 80
	RF: r2 = 0.956, rmse = 235.577
	ET: r2 = 1.000, rmse = 0.216
	TF: r2 = 0.911, rmse = 335.348
run = 81
	RF: r2 = 0.963, rmse = 215.197
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.911, rmse = 333.941
run = 82
	RF: r2 = 0.981, rmse = 153.186
	ET: r2 = 1.000, rmse = 0.127
	TF: r2 = 0.909, rmse = 337.916
run = 83
	RF: r2 = 0.978, rmse = 166.313
	ET: r2 = 1.000, rmse = 0.043
	TF: r2 = 0.911, rmse = 335.183
run = 84
	RF: r2 = 0.963, rmse = 214.473
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.911, rmse = 334.997
run = 85
	RF: r2 = 0.978, rmse = 165.547
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.911, rmse = 333.640
run = 86
	RF: r2 = 0.945, rmse = 262.363
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.911, rmse = 334.443
run = 87
	RF: r2 = 0.983, rmse = 147.794
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.910, rmse = 337.362
run = 88
	RF: r2 = 0.964, rmse = 211.307
	ET: r2 = 1.000, rmse = 0.070
	TF: r2 = 0.911, rmse = 334.050
run = 89
	RF: r2 = 0.963, rmse = 214.942


## Prepare for submission

In [10]:
# target end dates
cur_dt = datetime.strptime(steps[cur_idx], '%Y-%m-%d')
fc_dt = (cur_dt + timedelta(days=1)).date()
nxt_dt_1 = (cur_dt + timedelta(days=7)).date()
nxt_dt_2 = (cur_dt + timedelta(days=14)).date()
nxt_dts = [nxt_dt_1, nxt_dt_2]

# quantiles
qs = [.025, .1, .25, .5, .75, .9, .975]

In [11]:
df_county_prd_1 = pd.DataFrame(data=np.array(prds_1).T, index=df.index, columns=range(num_runs))
df_county_prd_2 = pd.DataFrame(data=np.array(prds_2).T, index=df.index, columns=range(num_runs))
county_dfs = [df_county_prd_1, df_county_prd_2]

### County

In [12]:
forecast_dfs = []

for i, county_df in enumerate(county_dfs):
    w = i + 1
    csv_fname = f'results/{steps[cur_idx]}_county_case_{w}-wk_{num_runs}.csv'
    county_df.to_csv(csv_fname, float_format='%.2f')
    
    # ========================= Point esitmate =========================START
    df_point = county_df.mean(axis=1).reset_index()\
        .rename(columns={0: 'value', 'fips': 'location'})
    df_point['type'] = 'point'
    df_point['quantile'] = 'NA'
    # ========================= Point esitmate =========================END

    # ========================= Quantile esitmates =========================START
    df_quantile = county_df.quantile(qs, axis=1).T.reset_index()\
        .melt(id_vars=['fips']).sort_values('fips')\
        .rename(columns={'fips': 'location', 'variable': 'quantile'})
    df_quantile['type'] = 'quantile'
    # ========================= Quantile esitmates =========================END

    df_forecast = pd.concat([df_point, df_quantile])
    df_forecast['target'] = f'{w} wk ahead inc case'
    df_forecast['target_end_date'] = nxt_dts[i]
    
    forecast_dfs.append(df_forecast)

df_forecast_county = pd.concat(forecast_dfs, axis=0).reset_index(drop=True)
print(df_forecast_county.shape)
df_forecast_county.head()

(49504, 6)


,location,quantile,type,value,target,target_end_date
0,01001,NA,point,205.146886,1 wk ahead inc case,2020-11-28
1,01003,NA,point,578.569519,1 wk ahead inc case,2020-11-28
2,01005,NA,point,31.742897,1 wk ahead inc case,2020-11-28
3,01007,NA,point,65.601499,1 wk ahead inc case,2020-11-28
4,01009,NA,point,186.931761,1 wk ahead inc case,2020-11-28


### States

In [13]:
forecast_dfs = []

for i, county_df in enumerate(county_dfs):
    w = i + 1
    
    df_tmp = county_df.reset_index()
    df_tmp['state'] = df_tmp['fips'].apply(lambda x: x[:2])
    state_df = df_tmp.groupby(by='state').sum()
    
    # ========================= Point esitmate =========================START
    df_point = state_df.mean(axis=1).reset_index()\
        .rename(columns={0: 'value', 'state': 'location'})
    df_point['type'] = 'point'
    df_point['quantile'] = 'NA'
    # ========================= Point esitmate =========================END

    # ========================= Quantile esitmates =========================START
    df_quantile = state_df.quantile(qs, axis=1).T.reset_index()\
        .melt(id_vars=['state']).sort_values('state')\
        .rename(columns={'state': 'location', 'variable': 'quantile'})
    df_quantile['type'] = 'quantile'
    # ========================= Quantile esitmates =========================END

    df_forecast = pd.concat([df_point, df_quantile])
    df_forecast['target'] = f'{w} wk ahead inc case'
    df_forecast['target_end_date'] = nxt_dts[i]
    
    forecast_dfs.append(df_forecast)

df_forecast_state = pd.concat(forecast_dfs, axis=0).reset_index(drop=True)
df_forecast_state.head()

,location,quantile,type,value,target,target_end_date
0,01,NA,point,16164.500262,1 wk ahead inc case,2020-11-28
1,02,NA,point,4314.260893,1 wk ahead inc case,2020-11-28
2,04,NA,point,22339.654286,1 wk ahead inc case,2020-11-28
3,05,NA,point,12394.137219,1 wk ahead inc case,2020-11-28
4,06,NA,point,96406.258957,1 wk ahead inc case,2020-11-28


### Nation

In [14]:
forecast_dfs = []

for i, county_df in enumerate(county_dfs):
    w = i + 1
    
    nation_df = pd.DataFrame(data=county_df.sum(axis=0), columns=['US']).T
    nation_df.index.name = 'nation'
    
    # ========================= Point esitmate =========================START
    df_point = nation_df.mean(axis=1).reset_index()\
        .rename(columns={0: 'value', 'nation': 'location'})
    df_point['type'] = 'point'
    df_point['quantile'] = 'NA'
    # ========================= Point esitmate =========================END

    # ========================= Quantile esitmates =========================START
    df_quantile = nation_df.quantile(qs, axis=1).T.reset_index()\
        .melt(id_vars=['nation']).sort_values('nation')\
        .rename(columns={'nation': 'location', 'variable': 'quantile'})
    df_quantile['type'] = 'quantile'
    # ========================= Quantile esitmates =========================END

    df_forecast = pd.concat([df_point, df_quantile])
    df_forecast['target'] = f'{w} wk ahead inc case'
    df_forecast['target_end_date'] = nxt_dts[i]
    
    forecast_dfs.append(df_forecast)

df_forecast_nation = pd.concat(forecast_dfs, axis=0).reset_index(drop=True)
df_forecast_nation

,location,quantile,type,value,target,target_end_date
0,US,NA,point,1.274484e+06,1 wk ahead inc case,2020-11-28
1,US,0.025,quantile,1.267036e+06,1 wk ahead inc case,2020-11-28
2,US,0.1,quantile,1.269649e+06,1 wk ahead inc case,2020-11-28
3,US,0.25,quantile,1.272020e+06,1 wk ahead inc case,2020-11-28
4,US,0.5,quantile,1.274935e+06,1 wk ahead inc case,2020-11-28
5,US,0.75,quantile,1.277231e+06,1 wk ahead inc case,2020-11-28
6,US,0.9,quantile,1.279022e+06,1 wk ahead inc case,2020-11-28
7,US,0.975,quantile,1.280772e+06,1 wk ahead inc case,2020-11-28
8,US,NA,point,1.364686e+06,2 wk ahead inc case,2020-12-05
9,US,0.025,quantile,1.351948e+06,2 wk ahead inc case,2020-12-05


In [15]:
df_forecast = pd.concat([df_forecast_county, df_forecast_state, df_forecast_nation])
df_forecast.reset_index(drop=True, inplace=True)
df_forecast['forecast_date'] = fc_dt
cols = ['forecast_date', 'target', 'target_end_date', 'location', 'type', 'quantile', 'value']
df_forecast = df_forecast[cols]

# =============== make nagative value zero ==============
value_dezero = df_forecast['value']
value_dezero[value_dezero < 0]  = 0
df_forecast['value'] = value_dezero

In [16]:
team = 'UChicagoCHATTOPADHYAY'
model = 'UnIT'
df_forecast.to_csv(f'results/{fc_dt}-{team}-{model}_case.csv', float_format='%.2f', index=False)

In [17]:
validation = '/home/yihuang/Documents/Data/covid19-forecast-hub/code/validation/validate_single_forecast_file.py'

! python3 {validation} results/2020-11-08-UChicagoCHATTOPADHYAY-UnIT_case.csv


VALIDATING results/2020-11-08-UChicagoCHATTOPADHYAY-UnIT_case.csv
✓ results/2020-11-08-UChicagoCHATTOPADHYAY-UnIT_case.csv is valid with no errors
